# Web scraping to find best deals at the best time
- Uses Selenium to scrape popular deals website

To do:
- Fix time column

In [255]:
import os  
from selenium import webdriver  
from selenium.webdriver.chrome.options import Options  
from selenium.webdriver.common.keys import Keys
import pandas as pd
import datetime as dt
import numpy as np
import datefinder
import dateparser
import seaborn as sns

In [155]:
# Set headless browser
options = Options()
options.headless = True

In [230]:
def clean_date(date_entry):
    temp = date_entry.split()
    if temp[-1]=='ago':
        # return dt.datetime.today().strftime("%d/%m/%Y")
        return pd.to_datetime('today')
    else:
        return date_entry

def clean_company(company_entry):
    # reverse split to split by last space symbol
    return company_entry.rsplit(' ', 1)[0]


def clean_price(price_entry):
    temp = price_entry[1:]
    temp = temp.replace(',', '')
    try:
        return round(float(temp))
    except ValueError:
        return np.nan


def clean_heat(heat_entry):
    temp = heat_entry[:-1]
    try:
        return int(temp)
    except ValueError:
        return np.nan

In [250]:
df = pd.DataFrame(columns=['Item', 'Price', 'Company', 'Heat', 'Date'])

for page_no in range(1,11):
    driver = webdriver.Chrome('chromedriver.exe', chrome_options=options)
    driver.get('https://www.hotukdeals.com/tag/electronics?page='+str(page_no))
    item_data = driver.find_elements_by_class_name('threadGrid-title')  # get title, price, company
    forum_data = results = driver.find_elements_by_class_name('threadGrid-headerMeta')  # get heat, date

    df_item = pd.DataFrame(columns=['Item', 'Price', 'Company'])
    for i in range(len(item_data)):
        temp = item_data[i].text.split('\n')
        df_item.loc[i] = [temp[0], temp[1], temp[-1]]

    df_forum = pd.DataFrame(columns=['Heat', 'Date'])
    for i in range(len(forum_data)):
        temp = forum_data[i].text.split('\n')
        df_forum.loc[i] = [temp[0], temp[-1]]

    df_temp = pd.concat([df_item, df_forum], axis=1)
    df = pd.concat([df, df_temp], axis=0)

# Clean price
df['Price'] = df['Price'].map(lambda x: clean_price(x))
# Clean company
df['Company'] = df['Company'].map(lambda x: clean_company(x))
# Clean date
df['Date'] = df['Date'].map(lambda x: clean_date(x))
# Clean heat
df['Heat'] = df['Heat'].map(lambda x: clean_heat(x))
# Drop NAN
df = df.dropna()
df = df.reset_index(drop=True)

In [259]:
df

,Item,Price,Company,Heat,Date
0,Mpow 35Hrs Noise Cancelling Headphones £19.99 ...,20.0,Amazon,131.0,2020-10-12 11:53:49.690404
1,Kingston A2000 (SA2000M8/1000G) SSD NVMe PCIe ...,88.0,Amazon Germany,151.0,2020-10-12 11:53:49.691401
2,Boltune BT-BH024 Bluetooth 5.0 earphones with ...,24.0,Amazon,135.0,2020-10-12 11:53:49.691401
3,"MSI X470 GAMING PLUS MAX Motherboard ATX, AM4,...",75.0,Amazon,317.0,2020-10-12 11:53:49.691401
4,Sony A7R IV Mirrorless Camera Body £2999 at Ca...,2999.0,CameraWorld,132.0,2020-10-12 11:53:49.691401
...,...,...,...,...,...
190,TP-Link Deco E4 (3-Pack) Dual Band AC1200 Mesh...,89.0,eBay,303.0,Made hot 8th Oct
191,SanDisk Cruzer FIT USB Flash Drive 32GB/64GB f...,5.0,PicStop,351.0,Made hot 8th Oct
192,LG 55 Inch 55OLED55BX Smart 4K Ultra HD OLED T...,1099.0,Argos,499.0,Made hot 8th Oct
193,Grade A Refurbished OnePlus NORD - 128GB Unloc...,297.0,eBay,178.0,Made hot 8th Oct
